# Developing a Classifier for Movie Rating for IMDB Data Set

## Section 0: Load Dependencies and Data

In [ ]:
import pandas as pd  # For data manipulation and analysis
from collections import Counter  # For counting hashable objects
from sklearn.decomposition import PCA  # For dimensionality reduction
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For visualization
from sklearn.feature_selection import mutual_info_classif  # For feature importance estimation
from sklearn.linear_model import LogisticRegression  # Logistic Regression model
from sklearn.svm import SVC  # Support Vector Classifier for SVM
from sklearn.model_selection import KFold  # For k-fold cross-validation
import statistics as stat  # For statistical calculations
from sklearn.preprocessing import StandardScaler  # For standardizing features
from sklearn.cross_decomposition import PLSRegression  # Partial Least Squares regression
from sklearn.metrics import r2_score, accuracy_score, classification_report, confusion_matrix  # For calculating R-squared metric
import random  # For generating random numbers
from sklearn.model_selection import GridSearchCV  # For hyperparameter optimization
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# Loading the datasets for training and testing
try:
    Test = pd.read_csv('project_data/test_dataset.csv')  # Test dataset
    Train = pd.read_csv('project_data/train_dataset.csv')  # Training dataset
    print("Datasets loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: {e}. Ensure the file paths are correct.")
    raise
except pd.errors.EmptyDataError as e:
    print(f"Error: {e}. Ensure the CSV files are not empty.")
    raise
except Exception as e:
    print(f"An unexpected error occurred while loading datasets: {e}")
    raise



# Validating the datasets
def validate_dataset(df, name):
    print(f"\nValidating {name} dataset:")
    # Check for missing values
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print("Warning: Missing values detected.")
        print(missing_values[missing_values > 0])
    else:
        print("No missing values detected.")
    
    # Check for duplicate rows
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        print(f"Warning: {duplicate_count} duplicate rows found.")
    else:
        print("No duplicate rows detected.")
    
    # Check for feature consistency
    print("Dataset shape:", df.shape)
    if df.empty:
        print(f"Error: {name} dataset is empty.")
        raise ValueError(f"{name} dataset is empty.")
    elif len(df.columns) == 0:
        print(f"Error: {name} dataset has no columns.")
        raise ValueError(f"{name} dataset has no columns.")
    else:
        print(f"{name} dataset structure is valid.")
    
    # Display basic statistics for numeric columns
    print("Basic statistics:")
    print(df.describe())

# Validate both Train and Test datasets
validate_dataset(Train, "Training")
validate_dataset(Test, "Testing")

print("\nValidation complete.")


In [ ]:
import numpy as np
import pandas as pd


# Load plot keyword embeddings (Doc2Vec representation)
KWEembeddings = np.load("project_data/features_doc2vec/features_doc2vec/train_doc2vec_features_plot_keywords.npy")

# Load genre embeddings (Doc2Vec representation)
Gembeddings = np.load("project_data/features_doc2vec/features_doc2vec/train_doc2vec_features_genre.npy")

# Load title embeddings (FastText representation)
Tembeddings = np.load("project_data/features_fasttext/features_fasttext/train_fasttext_title_embeddings.npy")


KeyWordEmbeddings = pd.DataFrame(KWEembeddings).add_prefix("KWE_")
GenreEmbeddings    = pd.DataFrame(Gembeddings).add_prefix("GenEmb_")
TitleEmbeddings    = pd.DataFrame(Tembeddings).add_prefix("TitleEmb_")


print("Shape of KeyWordEmbeddings:", KeyWordEmbeddings.shape)
print("Shape of GenreEmbeddings:", GenreEmbeddings.shape)
print("Shape of TitleEmbeddings:", TitleEmbeddings.shape)



## Section 1: Pre-processing, and exploratory analysis

### 1.1: Analyzing Genre Distribution and Frequency

In [ ]:
# Split the "genres" column into a list of genres for each movie
Train["genres"] = Train["genres"].apply(lambda x: x.split("|"))

# Initialize an empty list to store all genre occurrences
genreFreqList = []

# Flatten the nested list of genres to prepare for frequency calculation
for genres in Train["genres"]:
    for genre in genres:
        genreFreqList.append(genre)

# Count the occurrences of each genre using Counter
count = Counter(genreFreqList)
print("Genre Frequency Count:", count)  # Output the genre frequency count
print("Number of unique genres:", len(count))  # Output the number of unique genres

# Function to one-hot encode genres for each row
def OneHotEncodeGenre(row):
    """
    Updates the row's genre columns with one-hot encoding.
    Each genre column value is incremented by 1 if the genre is present in the row.

    Parameters:
    row (Series): A row of the DataFrame containing a list of genres.

    Returns:
    Series: Updated row with one-hot encoded genre columns.
    """
    for genre in row["genres"]:  # Iterate over genres in the current row
        row[genre] += 1  # Increment the respective genre column
    return row

# Get the list of unique genres as column labels
labels = list(count.keys())

# Create a DataFrame with one column per genre, initialized to 0
EncodedGenres = pd.DataFrame(0, index=np.arange(len(Train)), columns=labels)

# Copy the "genres" column from the original Train dataset to preserve genre lists
EncodedGenres["genres"] = Train["genres"].copy(deep=True)

print("Initial one-hot encoded DataFrame (before processing):")
print(EncodedGenres)

# Apply the one-hot encoding function to each row of the DataFrame
EncodedGenres = EncodedGenres.apply(lambda x: OneHotEncodeGenre(x), axis=1)

# Drop the now-unnecessary "genres" column as the data has been encoded
EncodedGenres = EncodedGenres.drop(columns=["genres"])

# Output the final one-hot encoded DataFrame
print("Final one-hot encoded DataFrame (after processing):")
print(EncodedGenres)


### 1.3 Content Rating Transformation

In [ ]:
from collections import Counter

# Extract content ratings from the dataset
ratings = Train["content_rating"]

# Generate a frequency list of ratings for analysis
ratingsFreqList = [rating for rating in Train["content_rating"]]

# Count the occurrences of each rating
count = Counter(ratingsFreqList)
print("Ratings Frequency Count:", count)

# Define a function to convert content ratings into numerical categories
def convertRatings(row):
    """
    Convert content ratings to numerical categories:
    1 - G, Approved, GP, Passed
    2 - PG, PG-13, or any rating containing '13'
    3 - M, Not Rated, Unrated
    4 - R
    5 - X, NC-17, or similar

    Parameters:
    row (str): The content rating.

    Returns:
    int: The numerical category for the rating.
    """
    # Ensure the input is a string to avoid errors
    row = str(row)

    # Map ratings to categories
    if 'pg' in row.lower() or '13' in row.lower(): 
        return 2
    if row.lower() in ['g', 'gp', 'passed']:
        return 1
    if 'm' in row.lower() or row.lower() in ["not rated", "unrated"]:
        return 3
    if row.lower() == "r":
        return 4
    if row.lower() in ["x", "nc-17"] or "nc" in row.lower():
        return 5
    # Default to category 3 if no specific match
    return 3

# Apply the conversion function to the ratings
ratings = ratings.apply(lambda x: convertRatings(x))

# Print the transformed ratings for validation
print("Transformed Ratings (First 10):")
print(ratings.head(10))


### 1.4 Country Mapping and One-Hot Encoding of Continents

In [ ]:
# Get frequencies of countries in the dataset
countryFreqList = [i for i in Train["country"]]
country_count = Counter(countryFreqList)
print("Country Frequency Count:", country_count)

# Prepare a DataFrame to map countries to sub-regions and continents
cont = Train["country"]
sub_region = pd.DataFrame("BLANK", index=np.arange(len(cont)), columns=["sub_region", "continent"])
cont = pd.concat([cont, sub_region], axis=1)

# Load a country-to-region mapping table
countryTable = pd.read_csv("CountryToRegion.csv")

# Function to map countries to their corresponding sub-region and continent
def getRegion(row):
    """
    Maps a country to its sub-region and continent based on the provided mapping table.
    Defaults to 'Northern America' for sub-region and 'Americas' for continent if no match is found.
    """
    val = countryTable.loc[countryTable["country"] == row["country"]]
    if len(val) == 0:
        row["sub_region"] = "Northern America"  # Default sub-region
        row["continent"] = "Americas"  # Default continent
        return row
    # Assign sub-region and continent from the mapping table
    row["sub_region"] = val.iloc[0]["sub_region"]
    row["continent"] = val.iloc[0]["continent"]
    return row

# Apply the region-mapping function to the DataFrame
cont = cont.apply(lambda x: getRegion(x), axis=1)

# Function for one-hot encoding continents
def OneHotEncodeSingleCont(row):
    """
    One-hot encodes a single continent for a given row.
    """
    row[row["continent"]] += 1
    return row

# Get frequencies of continents
continentFreqList = [value for value in cont["continent"]]
continent_count = Counter(continentFreqList)
print("Continent Frequency Count:", continent_count)

# Create a DataFrame for one-hot encoding continents
labels = list(continent_count.keys())  # Get unique continent labels
EncodedCont = pd.DataFrame(0, index=np.arange(len(Train)), columns=labels)  # Initialize one-hot columns
EncodedCont["continent"] = cont["continent"].copy(deep=True)

# Apply one-hot encoding to each row
EncodedCont = EncodedCont.apply(lambda x: OneHotEncodeSingleCont(x), axis=1)

# Drop the original "continent" column as it is no longer needed
EncodedCont = EncodedCont.drop(columns=["continent"])

# Display the one-hot encoded continent DataFrame for verification
print("One-Hot Encoded Continent DataFrame:")
print(EncodedCont.head())


### 1.5: Combining and Scaling Features

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


EncodedCont = EncodedCont.reindex(Train.index)
ratings = ratings.reindex(Train.index)
TitleEmbeddings = TitleEmbeddings.reindex(Train.index)
KeyWordEmbeddings = KeyWordEmbeddings.reindex(Train.index)
GenreEmbeddings = GenreEmbeddings.reindex(Train.index)
EncodedGenres = EncodedGenres.reindex(Train.index)

numericColumnSet = [
    'num_critic_for_reviews',
    'duration',
    'director_facebook_likes',
    'actor_3_facebook_likes',
    'actor_1_facebook_likes',
    'gross',
    'num_voted_users',
    'cast_total_facebook_likes',
    'facenumber_in_poster',
    'num_user_for_reviews',
    'title_year',
    'actor_2_facebook_likes',
    'movie_facebook_likes',
    'average_degree_centrality'
]


# PreTotalData = numeric + continents + ratings
PreTotalData = pd.concat([Train[numericColumnSet], EncodedCont, ratings], axis=1)
PreTotalData.columns = PreTotalData.columns.astype(str)  # ensure column names are strings

# TotalData = numeric + continents + ratings + TitleEmbeddings + KeyWordEmbeddings
TotalData = pd.concat(
    [Train[numericColumnSet], EncodedCont, ratings, TitleEmbeddings, KeyWordEmbeddings],
    axis=1
)
TotalData.columns = TotalData.columns.astype(str)

# TotalDataPlusGenreEmbeddings = TotalData + GenreEmbeddings
TotalDataPlusGenreEmbeddings = pd.concat([TotalData, GenreEmbeddings], axis=1)
TotalDataPlusGenreEmbeddings.columns = TotalDataPlusGenreEmbeddings.columns.astype(str)

# TotalDataPlusGenreEncodings = TotalData + EncodedGenres
TotalDataPlusGenreEncodings = pd.concat([TotalData, EncodedGenres], axis=1)
TotalDataPlusGenreEncodings.columns = TotalDataPlusGenreEncodings.columns.astype(str)


scaler = StandardScaler()

# Scale PreTotalData
S_TotalData_array = scaler.fit_transform(PreTotalData)
S_TotalData = pd.DataFrame(S_TotalData_array, columns=PreTotalData.columns)

# Scale TotalDataPlusGenreEmbeddings
scaled_embeddings_data = scaler.fit_transform(TotalDataPlusGenreEmbeddings)
S_TotalDataPlusGenreEmbeddings = pd.DataFrame(
    scaled_embeddings_data,
    columns=TotalDataPlusGenreEmbeddings.columns
)

# Scale TotalDataPlusGenreEncodings
scaled_encoding_data = scaler.fit_transform(TotalDataPlusGenreEncodings)
S_TotalDataPlusGenreEncoding = pd.DataFrame(
    scaled_encoding_data,
    columns=TotalDataPlusGenreEncodings.columns
)

# Verify Shapes
print("Shape of S_TotalData:", S_TotalData.shape)
print("Shape of S_TotalDataPlusGenreEmbeddings (scaled):", S_TotalDataPlusGenreEmbeddings.shape)
print("Shape of S_TotalDataPlusGenreEncoding (scaled):", S_TotalDataPlusGenreEncoding.shape)


y = Train["imdb_score_binned"]  # the target

# Split for S_TotalData
X_train_S_TotalData, X_test_S_TotalData, y_train_S_TotalData, y_test_S_TotalData = train_test_split(
    S_TotalData,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Split for S_TotalDataPlusGenreEmbeddings
X_train_S_TotalDataPlusGenreEmbeddings, X_test_S_TotalDataPlusGenreEmbeddings, y_train_S_TotalDataPlusGenreEmbeddings, y_test_S_TotalDataPlusGenreEmbeddings = train_test_split(
    S_TotalDataPlusGenreEmbeddings,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Split for S_TotalDataPlusGenreEncoding
X_train_S_TotalDataPlusGenreEncoding, X_test_S_TotalDataPlusGenreEncoding, y_train_S_TotalDataPlusGenreEncoding, y_test_S_TotalDataPlusGenreEncoding = train_test_split(
    S_TotalDataPlusGenreEncoding,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("S_TotalData train shape:", X_train_S_TotalData.shape, "test shape:", X_test_S_TotalData.shape)
print("S_TotalDataPlusGenreEmbeddings train shape:", X_train_S_TotalDataPlusGenreEmbeddings.shape,
      "test shape:", X_test_S_TotalDataPlusGenreEmbeddings.shape)
print("S_TotalDataPlusGenreEncoding train shape:", X_train_S_TotalDataPlusGenreEncoding.shape,
      "test shape:", X_test_S_TotalDataPlusGenreEncoding.shape)

# Check data types if desired
print("\nType of X_train_S_TotalDataPlusGenreEncoding:", type(X_train_S_TotalDataPlusGenreEncoding))
print("Type of y_train_S_TotalDataPlusGenreEncoding:", type(y_train_S_TotalDataPlusGenreEncoding))


## Section 2: Feature Engineering and Dimensionality Reduction

### 2.1: Principal Component Analysis (PCA) on Keyword Embeddings

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA

# Perform PCA on the KeyWordEmbeddings
pca = PCA()
pca.fit(KeyWordEmbeddings)  # Fit PCA model on the embeddings

# Extract explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# Set up the plot aesthetics using seaborn
sns.set_theme(style="whitegrid")

# Create the figure and axis objects for cumulative variance plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plot cumulative variance with markers
ax.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
        marker='o', linestyle='--', label='Cumulative Explained Variance', color='blue')

# Highlight the point where 95% variance is explained
components_95 = np.argmax(cumulative_variance >= 0.95) + 1
ax.axhline(y=0.95, color='red', linestyle='-', label='95% Variance Threshold')
ax.axvline(x=components_95, color='green', linestyle='--', 
           label=f'{components_95} Components for 95% Variance')

# Add annotations for clarity
ax.annotate(f'{components_95} Components', 
            xy=(components_95, 0.95), 
            xytext=(components_95 + 5, 0.85),
            arrowprops=dict(facecolor='black', arrowstyle='->'),
            fontsize=12)

# Customize the plot
ax.set_title('Explained Variance vs. Number of Principal Components', fontsize=16)
ax.set_xlabel('Number of Principal Components', fontsize=14)
ax.set_ylabel('Cumulative Explained Variance', fontsize=14)
ax.set_xlim(1, len(cumulative_variance))
ax.set_ylim(0.0, 1.05)
ax.legend(loc='lower right', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

# Heatmap to visualize contribution of principal components
contribution_matrix = pca.components_[:components_95]
plt.figure(figsize=(12, 8))
sns.heatmap(contribution_matrix, cmap='coolwarm', center=0, 
            annot=False, fmt=".2f", xticklabels=False, yticklabels=True)
plt.title(f'Contribution of First {components_95} Principal Components', fontsize=16)
plt.xlabel('Features', fontsize=14)
plt.ylabel('Principal Components', fontsize=14)
plt.tight_layout()
plt.show()


### 2.2 Feature Evaluation and PCA on Genre Embeddings

In [ ]:
# Evaluate mutual information between title embeddings and binned IMDb scores
print("Mutual Information between Title Embeddings and IMDb Score Bins:")
mi_scores = mutual_info_classif(TitleEmbeddings, Train["imdb_score_binned"])
print(mi_scores)

# Calculate mutual information scores
mi_scores = mutual_info_classif(TitleEmbeddings, Train["imdb_score_binned"])

# Create a DataFrame for better visualization
mi_df = pd.DataFrame({'Feature': [f'Feature_{i+1}' for i in range(len(mi_scores))], 'MI Score': mi_scores})

# Sort the DataFrame by MI scores in descending order for better insights
mi_df = mi_df.sort_values(by='MI Score', ascending=False)

# Plot the mutual information scores as a bar chart
plt.figure(figsize=(12, 6))
plt.bar(mi_df['Feature'], mi_df['MI Score'], color='skyblue')

# Add plot labels and title
plt.xlabel('Features', fontsize=12)
plt.ylabel('Mutual Information Score', fontsize=12)
plt.title('Mutual Information Scores for Title Embeddings', fontsize=14)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display the plot
plt.tight_layout()
plt.show()

print('\n\n\n')

# Perform PCA on GenreEmbeddings
pca = PCA()
pca.fit(GenreEmbeddings)

# Print the explained variance ratio for PCA
print("Explained Variance Ratio for Genre Embeddings PCA:")
print(pca.explained_variance_ratio_)

# Plot cumulative explained variance for GenreEmbeddings
plt.rcParams["figure.figsize"] = (12, 6)  # Set the figure size

fig, ax = plt.subplots()

# x-axis: Component indices (1-based for human readability)
xi = np.arange(1, len(pca.explained_variance_ratio_) + 1, step=1)

# y-axis: Cumulative sum of explained variance ratios
y = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance
plt.ylim(0.0, 1.1)  # Set y-axis range
plt.plot(xi, y, marker='o', linestyle='--', color='b', label='Cumulative Variance')

# Add plot labels and title
plt.xlabel('Number of Components')
plt.xticks(np.arange(0, len(pca.explained_variance_ratio_) + 1, step=10))  # Adjust step size if needed
plt.ylabel('Cumulative Variance (%)')
plt.title('The Number of Components Needed to Explain Variance')

# Add reference line for the 95% variance threshold
plt.axhline(y=0.95, color='r', linestyle='-', label='95% Threshold')
plt.text(10, 0.88, '95% cut-off threshold', color='red', fontsize=12)

# Add gridlines and legend for clarity
ax.grid(axis='x')
plt.legend()

# Display the plot
plt.show()

### 2.3: Partial Least Squares (PLS) Regression and Component Analysis

In [ ]:
# Standardize the features and target variable
X_scaled = S_TotalDataPlusGenreEmbeddings
y_scaled = Train["imdb_score_binned"].values.reshape(-1, 1).ravel()

# Fit PLS regression with standardized data
pls = PLSRegression(n_components=len(S_TotalDataPlusGenreEmbeddings.columns) - 1, max_iter=100000)
pls.fit(X_scaled, y_scaled)

# Evaluate R² for individual components and overall performance
def pls_explained_variance(pls, X, Y_true, do_plot=True):
    r2 = np.zeros(pls.n_components)
    x_transformed = pls.transform(X)
    for i in range(0, pls.n_components):
        Y_pred = (np.dot(x_transformed[:, i][:, np.newaxis],
                         pls.y_loadings_[:, i][:, np.newaxis].T) * pls._y_std
                  + pls._y_mean)
        r2[i] = r2_score(Y_true, Y_pred)
    
    overall_r2 = r2_score(Y_true, pls.predict(X))

    if do_plot:
        import seaborn as sns
        sns.set(style="whitegrid")
        plt.figure(figsize=(12, 6))
        sns.lineplot(x=np.arange(1, pls.n_components + 1), y=r2, marker='o', label='Individual R²')
        plt.axhline(overall_r2, color='r', linestyle='--', label=f'Overall R²: {overall_r2:.3f}')
        plt.xlabel('PLS Component #', fontsize=12)
        plt.ylabel('R²', fontsize=12)
        plt.title(f'Summed Individual R²: {np.sum(r2):.3f}, Overall R²: {overall_r2:.3f}', fontsize=14)
        plt.legend()
        plt.show()

    return r2, overall_r2

# Evaluate and plot explained variance
r2, overall_r2 = pls_explained_variance(pls, X_scaled, y_scaled, do_plot=True)

# Find the optimal number of PLS components using cross-validation
best_r2 = float('-inf')
best_n_components = 0

for n in range(1, 40):  # Test components from 1 to 20
    pls = PLSRegression(n_components=n)
    scores = cross_val_score(pls, X_scaled, y_scaled, cv=5, scoring='r2')
    mean_r2 = np.mean(scores)
    # print(f"Components: {n}, Mean R²: {mean_r2:.3f}")
    if mean_r2 > best_r2:
        best_r2 = mean_r2
        best_n_components = n

print(f"Best number of components: {best_n_components}, Best R²: {best_r2:.3f}")

# Fit PLS with the optimal number of components
pls = PLSRegression(n_components=best_n_components)
pls.fit(X_scaled, y_scaled)
r2, overall_r2 = pls_explained_variance(pls, X_scaled, y_scaled, do_plot=True)


# Calculate cumulative variance for selected components
PLScomponents = pls.transform(X_scaled)
tempList = []
for i in range(1, best_n_components + 1):
    Itemp = PLScomponents[:, i - 1]
    Jtemp = y_scaled
    tempList.append(np.abs(np.cov(Itemp, Jtemp)[1, 0]))

# Plot cumulative variance explained by components
plt.figure(figsize=(12, 6))
sns.set(style="whitegrid")
xi = np.arange(1, best_n_components + 1, step=1)
y = np.cumsum(tempList) / sum(tempList)

# Calculate cumulative variance for selected components
PLScomponents = pls.transform(X_scaled)
tempList = []
for i in range(1, best_n_components + 1):
    Itemp = PLScomponents[:, i - 1]
    Jtemp = y_scaled
    tempList.append(np.abs(np.cov(Itemp, Jtemp)[1, 0]))

# Plot cumulative variance explained by components
plt.figure(figsize=(12, 6))
sns.set(style="whitegrid")
xi = np.arange(1, best_n_components + 1, step=1)
y = np.cumsum(tempList) / sum(tempList)

n_comp_95 = np.argmax(y >= 0.95) + 1

# Existing plot code
sns.lineplot(x=xi, y=y, marker='o', linestyle='--', color='b', label='Cumulative Variance')
plt.axhline(y=0.95, color='r', linestyle='-', label='95% Cut-off Threshold')

# Existing vertical line/text for best_n_components
plt.vlines(best_n_components, 0, 1.2, linestyles='dotted', colors='gray', 
           label=f'Selected Components ({best_n_components})')
plt.text(best_n_components + 0.5, 0.5, f'{best_n_components} Best R² Components', 
         color='black', fontsize=10)

plt.vlines(n_comp_95, 0, 1.2, linestyles='--', colors='green',
           label=f'95% Variance at {n_comp_95} Components')
plt.text(n_comp_95 + 0.5, 0.4, f'{n_comp_95} Components for 95% Var',
         color='green', fontsize=10)

plt.ylim(0.0, 1.1)
plt.xlabel('Number of Components', fontsize=12)
plt.ylabel('Cumulative Variance (%)', fontsize=12)
plt.title('Cumulative Variance Explained by Selected Components', fontsize=14)
plt.legend()
plt.show()


## Section 3: Dataset Selection

### 3.1 Baseline Whole Dataset Evaluation with Cross-Validation

In [ ]:
class DatasetWrapper:
    def __init__(self, name, X_train, y_train, X_test, y_test):
        self.name = name
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

# Create your dataset wrappers (replace with your own variables)
datasets = [
    DatasetWrapper(
        "TotalData",
        X_train_S_TotalData,
        y_train_S_TotalData,
        X_test_S_TotalData,
        y_test_S_TotalData
    ),
    DatasetWrapper(
        "TotalDataPlusGenreEncodings",
        X_train_S_TotalDataPlusGenreEncoding,
        y_train_S_TotalDataPlusGenreEncoding,
        X_test_S_TotalDataPlusGenreEncoding,
        y_test_S_TotalDataPlusGenreEncoding
    ),
    DatasetWrapper(
        "TotalDataPlusGenreEmbeddings",
        X_train_S_TotalDataPlusGenreEmbeddings,
        y_train_S_TotalDataPlusGenreEmbeddings,
        X_test_S_TotalDataPlusGenreEmbeddings,
        y_test_S_TotalDataPlusGenreEmbeddings
    )
]

# Define the models to evaluate
models = [
    ("Logistic Regression", LogisticRegression(max_iter=100000)),
    ("Support Vector Classifier (SVC)", SVC())
]


# Function to evaluate models on multiple datasets with cross-validation
def evaluate_models_on_splits(datasets, models):
    """
    Evaluate multiple models on multiple train-test splits.

    Parameters:
    - datasets: List of DatasetWrapper objects, each containing X_train, y_train, X_test, y_test
    - models: List of (model_name, model_instance) tuples

    Prints out the accuracy on the test set for each dataset and each model.
    """
    for ds in datasets:
        print(f"\nEvaluating Dataset: {ds.name}")
        for model_name, model in models:
            # Fit the model on the train set
            model.fit(ds.X_train, ds.y_train)
            # Predict on the test set
            y_pred = model.predict(ds.X_test)
            # Calculate test accuracy
            accuracy = accuracy_score(ds.y_test, y_pred)
            print(f"  Model: {model_name} | Test Accuracy: {accuracy:.4f}")



# Run the evaluation
evaluate_models_on_splits(datasets, models)


### 3.2 Feature Selection Using Ablation Approach

In [ ]:
def accuracy_SVM(X, Y, Xt, Yt):
    """
    Trains an SVM on (X, Y) and returns accuracy on (Xt, Yt).
    """
    clf = SVC()
    clf.fit(X, Y)
    return clf.score(Xt, Yt)

def accuracy_Logistic(X, Y, Xt, Yt):
    """
    Trains a Logistic Regression on (X, Y) and returns accuracy on (Xt, Yt).
    """
    clf = LogisticRegression(max_iter=100000, random_state=1273213)
    clf.fit(X, Y)
    return clf.score(Xt, Yt)


def ablationFeatureSelection(fullDataSet, 
                             targetSeries, 
                             scoreFunc, 
                             MaxOrMin, 
                             nSplits, 
                             threshold=False):
    """
    Given a dataset (fullDataSet) and a target (targetSeries),
    performs 1-step 'ablation': 
      1) Compute the baseline score across K-folds 
      2) For each feature, drop it and measure the new score
      3) Return the feature whose removal yields the "best" score 
         (depending on MaxOrMin), along with that score.
    """
    kf = KFold(n_splits=nSplits, random_state=10201331, shuffle=True)

    # 1) Compute baseline
    baseline_scores = []
    for train_index, test_index in kf.split(fullDataSet):
        X_train = fullDataSet.iloc[train_index]
        Y_train = targetSeries.iloc[train_index]
        X_test  = fullDataSet.iloc[test_index]
        Y_test  = targetSeries.iloc[test_index]
        baseline_scores.append(scoreFunc(X_train, Y_train, X_test, Y_test))
    baseline = stat.mean(baseline_scores)
    print("Ablation baseline score:", baseline)

    # 2) Check the score for removing each feature
    feature_scores = {}
    for col in fullDataSet.columns:
        tmpData = fullDataSet.drop(columns=[col], axis=1)
        tmp_scores = []
        for train_index, test_index in kf.split(tmpData):
            X_train = tmpData.iloc[train_index]
            Y_train = targetSeries.iloc[train_index]
            X_test  = tmpData.iloc[test_index]
            Y_test  = targetSeries.iloc[test_index]
            tmp_scores.append(scoreFunc(X_train, Y_train, X_test, Y_test))
        feature_scores[col] = stat.mean(tmp_scores)

    # 3) Return the column that yields the "best" (max or min) score
    if MaxOrMin == 'maxamise':
        best_val = max(feature_scores.values())
        best_cols = [c for c, val in feature_scores.items() if val == best_val]
    else:
        best_val = min(feature_scores.values())
        best_cols = [c for c, val in feature_scores.items() if val == best_val]

    chosen_col = random.choice(best_cols)  # pick one if there's a tie
    return chosen_col, feature_scores[chosen_col]


def buildUp(currFeatureSet, 
            fullDataSet, 
            targetSeries, 
            scoreFunc, 
            MaxOrMin, 
            nSplits, 
            threshold=False):
    """
    Attempt to 'build up' features one-by-one:
      - Start from currFeatureSet (possibly empty)
      - For each column not yet in currFeatureSet, 
        add it and compute the cross-validation score
      - Return the column that yields the best improvement.
    """
    kf = KFold(n_splits=nSplits, random_state=10201331, shuffle=True)
    
    # Evaluate each feature *not* in currFeatureSet
    candidate_scores = {}
    for col in fullDataSet.columns:
        if col in currFeatureSet:
            continue
        combo = currFeatureSet + [col]
        tmp_scores = []
        for train_index, test_index in kf.split(fullDataSet):
            X_train = fullDataSet[combo].iloc[train_index]
            Y_train = targetSeries.iloc[train_index]
            X_test  = fullDataSet[combo].iloc[test_index]
            Y_test  = targetSeries.iloc[test_index]
            tmp_scores.append(scoreFunc(X_train, Y_train, X_test, Y_test))
        candidate_scores[col] = stat.mean(tmp_scores)

    # If there are no candidates left, return None
    if len(candidate_scores) == 0:
        return None, None

    # Identify the best feature to add
    if MaxOrMin == 'maxamise':
        best_val = max(candidate_scores.values())
        best_cols = [c for c, val in candidate_scores.items() if val == best_val]
    else:
        best_val = min(candidate_scores.values())
        best_cols = [c for c, val in candidate_scores.items() if val == best_val]

    chosen_col = random.choice(best_cols)
    return chosen_col, candidate_scores[chosen_col]

random.seed = 10023913  # Proper usage of random seed

ablationAttempt = S_TotalDataPlusGenreEncoding.copy(deep=True)
currFeat = []
baseLineAccuracy = 0

# BUILD UP STAGE
while True:
    columnName, accuracy = buildUp(currFeat,
                                   ablationAttempt,
                                   Train["imdb_score_binned"],
                                   accuracy_Logistic,
                                   'maxamise',
                                   5)
    if (columnName is None) or (accuracy is None):
        # No more features to add
        break
    if accuracy < baseLineAccuracy:
        # Stopping if there's no improvement
        break
    print(f"Adding feature: {columnName}, new accuracy = {accuracy}")
    currFeat.append(columnName)
    baseLineAccuracy = accuracy
    print("Current combo:", currFeat)

# ABLATION STAGE
baseline = baseLineAccuracy
while len(ablationAttempt[currFeat].columns) > 1:
    print("Ablation with feature set size:", len(currFeat))
    columnName, newScore = ablationFeatureSelection(
        ablationAttempt[currFeat], 
        Train["imdb_score_binned"], 
        accuracy_Logistic,
        'maxamise',
        5
    )
    if newScore < baseline:
        print(f"No improvement after trying to remove '{columnName}' (score {newScore} < baseline {baseline}). Stopping.")
        break
    print(f"Removing column: {columnName}, new ablation score = {newScore}")
    currFeat.remove(columnName)
    baseline = newScore

print("Final selected features:", currFeat)


### 3.3 Evaluation of Model on PLS Outcome

In [ ]:
kf = KFold(n_splits=20, random_state=10201331, shuffle=True)

avgScr = []
for train_index, test_index in kf.split(PLScomponents):
    # Slice the first n_comp_95 components for train and test
    X = PLScomponents[train_index, :n_comp_95]
    Y = Train["imdb_score_binned"].iloc[train_index]
    Xt = PLScomponents[test_index, :n_comp_95]
    Yt = Train["imdb_score_binned"].iloc[test_index]
    
    # Fit the Logistic Regression model
    clf = LogisticRegression(max_iter=100000)
    clf.fit(X, Y)
    
    # Evaluate on the test fold
    avgScr.append(clf.score(Xt, Yt))

# Print the mean accuracy across the 20 folds
print("Mean CV Accuracy:", stat.mean(avgScr))


## Section 4: Model Optimisation and Evaluation with Ablation Data Set

In [556]:
final_features = ['num_voted_users', 'Drama', 'title_year', 'gross', 'Action', 'Documentary', 'KWE_61', 'Asia', 'Animation', 'KWE_57', 'Musical', 'TitleEmb_10', 'KWE_3']

### 4.1 Logistic Regression

In [ ]:
X_train = X_train_S_TotalDataPlusGenreEncoding[final_features]
y_train = y_train_S_TotalDataPlusGenreEncoding

X_test = X_test_S_TotalDataPlusGenreEncoding[final_features]
y_test = y_test_S_TotalDataPlusGenreEncoding

# parameter grid
param_grid = [
    {
        'penalty': ['l1'],
        'solver': ['liblinear', 'saga'],  # only solvers that support l1
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'class_weight': [None]
    },
    {
        'penalty': ['l2'],
        'solver': ['liblinear', 'lbfgs', 'sag', 'saga'],  # solvers that support l2
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'class_weight': [None]
    }
]


log_reg = LogisticRegression(random_state=1273213, max_iter=100000)

grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    scoring='accuracy',   # or 'f1_macro', 'balanced_accuracy', etc.
    cv=5,                 # 5-fold cross-validation
    n_jobs=-1,            # use all available cores
    verbose=1             # print progress
)

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearch
print("Best parameters:", grid_search.best_params_)
print("Best CV accuracy:", grid_search.best_score_)


best_model = grid_search.best_estimator_  # best estimator from grid search

y_pred = best_model.predict(X_test)

# Accuracy
test_acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_acc)

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))  # set figure size
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix", fontsize=16)
plt.ylabel("True Label", fontsize=12)
plt.xlabel("Predicted Label", fontsize=12)

# Adjust tick label size (optional)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


### 4.2 Random Forest

In [ ]:
# Example param grid
param_grid = {
    'n_estimators': [100, 200, 300],      # Number of trees
    'max_depth': [None, 5, 10],          # Tree depth
    'max_features': ['sqrt', 'log2'],    # Features to consider at each split
    'min_samples_split': [2, 5],         # Min # of samples to split a node
    'min_samples_leaf': [1, 2, 4],       # Min # of samples at a leaf node
    'class_weight': [None, 'balanced'],  # Class imbalance handling
}

# Create a RandomForestClassifier (baseline model)
rf = RandomForestClassifier(random_state=42)

# Set up the grid search
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='accuracy',   # or 'f1_macro', etc.
    cv=5,                 # 5-fold cross-validation
    n_jobs=-1,            # use all available CPU cores
    verbose=1             # print progress messages
)

# Fit on the training set
grid_search.fit(X_train, y_train)

# Print out the best hyperparameters from the grid search
print("Best parameters found:", grid_search.best_params_)
print("Best CV accuracy:", grid_search.best_score_)

# Retrieve the best model from the grid search
best_rf = grid_search.best_estimator_

# Predict on the test set
y_pred = best_rf.predict(X_test)

# Evaluate
test_acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_acc)

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))  # set figure size
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix", fontsize=16)
plt.ylabel("True Label", fontsize=12)
plt.xlabel("Predicted Label", fontsize=12)

# Adjust tick label size (optional)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### 4.3 Gradient Boosting (XGBoost)

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------------------------------------------------------------
# 1) Prepare X_train, y_train, X_test, y_test
#    (Already created by your train-test split, for instance 80/20)
#    We assume these variables exist in your workspace.
# -------------------------------------------------------------------------
# X_train, y_train, X_test, y_test = ...

# If you have an extremely imbalanced dataset and your target is binary,
# it might be beneficial to look at 'f1', 'recall', or 'balanced_accuracy' 
# for scoring, in addition to (or instead of) accuracy.

# -------------------------------------------------------------------------
# 2) Define a parameter grid. Note scale_pos_weight for imbalance
# -------------------------------------------------------------------------
param_grid = {
    'n_estimators':      [100, 200, 300],
    'learning_rate':     [0.01, 0.1, 0.2],
    'max_depth':         [3, 5, 7],
    'subsample':         [0.8, 1.0],
    'colsample_bytree':  [0.8, 1.0],
    'min_child_weight':  [1, 3],
    'gamma':             [0, 1],
    # scale_pos_weight: number_of_negative_examples / number_of_positive_examples
    # If unsure, you can try a range of plausible values.
    'scale_pos_weight':  [1, 5, 10]
}

# -------------------------------------------------------------------------
# 3) Create and configure the XGBClassifier
#    - objective='binary:logistic' for binary classification
#    - use_label_encoder=False to avoid label-encoding warnings
#    - eval_metric='logloss' or 'auc' internally for xgboost
# -------------------------------------------------------------------------
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',  
    eval_metric='logloss',        
    use_label_encoder=False,
    random_state=42
)

# -------------------------------------------------------------------------
# 4) Set up the GridSearchCV
#    - Consider an appropriate scoring metric for imbalanced data
#      (e.g. 'f1', 'balanced_accuracy', 'roc_auc', etc.)
# -------------------------------------------------------------------------
grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='accuracy',     # or 'f1_macro', 'balanced_accuracy', 'roc_auc'
    cv=5,                   # 5-fold cross-validation
    n_jobs=-1,              # utilize all cores
    verbose=1               # print progress
)

# -------------------------------------------------------------------------
# 5) Fit the grid search on the training data
# -------------------------------------------------------------------------
grid_search.fit(X_train, y_train)

# Print best hyperparameters and CV score
print("Best parameters found:", grid_search.best_params_)
print("Best CV score:", grid_search.best_score_)

# -------------------------------------------------------------------------
# 6) Evaluate on the held-out test set
# -------------------------------------------------------------------------
best_xgb = grid_search.best_estimator_

# Predict on the test set
y_pred = best_xgb.predict(X_test)

# Calculate test accuracy (or use an alternative metric)
test_acc = accuracy_score(y_test, y_pred)
print("\nTest Accuracy:", test_acc)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Plot the confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("XGBoost - Confusion Matrix", fontsize=16)
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



### 4.4 Neural Network

In [ ]:
import sys
import torch
import torch.nn.functional as F
import torch.utils.data as dataUtils

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from collections import Counter

import matplotlib.pyplot as plt

import time


class NeuralNetwork(torch.nn.Module):
    def __init__(self, nInputs, l1, l2, l3, nClasses):
        super(NeuralNetwork, self).__init__()
        self.Numerical = torch.nn.Sequential(
            torch.nn.Linear(20, 10),
            torch.nn.Sigmoid(),
            torch.nn.Linear(10,10),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.25)
        )
        self.Embeddings1 = torch.nn.Sequential(
            torch.nn.Linear(100, 20),
            torch.nn.Sigmoid(),
            torch.nn.Linear(20,10),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.25)
        )
        self.Embeddings2 = torch.nn.Sequential(
            torch.nn.Linear(100,20),
            torch.nn.Sigmoid(),
            torch.nn.Linear(20,10),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.25)
        )
        self.Embeddings3 = torch.nn.Sequential(
            torch.nn.Linear(100,20),
            torch.nn.Sigmoid(),
            torch.nn.Linear(20,10),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.25)
        )
        self.Combination = torch.nn.Sequential(
            torch.nn.Linear(40,10),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.25),
            torch.nn.Linear(10,nClasses),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        a = self.Numerical(x[:,0:20])
        b = self.Embeddings1(x[:,20:120])
        c = self.Embeddings2(x[:,120:220])
        d = self.Embeddings3(x[:,220:320])
        combined = torch.cat((a,b,c,d), dim=1)
        out = self.Combination(combined)
        return out

def test(model, criterion, test_loader):
    test_loss = 0.
    test_preds, test_labels = list(), list()
    for i, data in enumerate(test_loader):
        x, labels = data

        with torch.no_grad():
            logits = model(x)
            predictions = torch.argmax(logits, dim=1)
            test_loss += criterion(input=logits, target=labels).item()
            test_preds.append(predictions)
            test_labels.append(labels)

    test_preds = torch.cat(test_preds)
    test_labels = torch.cat(test_labels)

    test_accuracy = torch.eq(test_preds, test_labels).float().mean().item()

    print('[TEST] Mean loss {:.4f} | Accuracy {:.4f}'.format(test_loss/len(test_loader), test_accuracy))


def train(model, train_loader, test_loader, optimizer, n_epochs=10):
    LOG_INTERVAL = 250
    running_loss, running_accuracy = list(), list()
    start_time = time.time()
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(n_epochs):

        epoch_loss = 0.

        for i, data in enumerate(train_loader):

            x, labels = data

            logits = model(x)

            predictions = torch.argmax(logits, dim=1)
            train_acc = torch.mean(torch.eq(predictions, labels).float()).item()

            loss = criterion(input=logits, target=labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_loss.append(loss.item())
            running_accuracy.append(train_acc)

            epoch_loss += loss.item()

            if i % LOG_INTERVAL == 0:
                deltaT = time.time() - start_time
                mean_loss = epoch_loss / (i+1)
                print('[TRAIN] Epoch {} [{}/{}]| Mean loss {:.4f} | Train accuracy {:.5f} | Time {:.2f} s'.format(epoch, 
                    i, len(train_loader), mean_loss, train_acc, deltaT))

        print('Epoch complete! Mean loss: {:.4f}'.format(epoch_loss/len(train_loader)))

        test(model, criterion, test_loader)
    return running_loss, running_accuracy


xTrain, xTest, yTrain, yTest = train_test_split(TotalDataPlusGenreEmbeddings, Train["imdb_score_binned"], test_size=0.2, random_state=981488)

xTrain = torch.tensor(xTrain.values, dtype=torch.float)
yTrain = torch.tensor(yTrain.values).type(torch.LongTensor)
trainData = dataUtils.TensorDataset(xTrain, yTrain)
trainLoader = dataUtils.DataLoader(trainData, batch_size=128, shuffle=True)

xTest = torch.tensor(xTest.values, dtype=torch.float)
yTest = torch.tensor(yTest.values).type(torch.LongTensor)
testData = dataUtils.TensorDataset(xTest, yTest)
testLoader = dataUtils.DataLoader(testData, batch_size=128, shuffle=False)

neuralNetwork = NeuralNetwork(len(TotalDataPlusGenreEmbeddings.columns), 150, 100, 50, 5)

optimizer = torch.optim.SGD(neuralNetwork.parameters(), lr=1e-2, momentum=0.5)

nnLoss, nnAcc = train(neuralNetwork, trainLoader, testLoader, optimizer, 100)

plt.plot(nnLoss)
plt.xlabel("iterations")
plt.ylabel("Cross-entropy Loss (Train)")
plt.show()

plt.plot(nnAcc)
plt.xlabel("iterations")
plt.ylabel("Accuracy (Train)")
plt.show()